In [1]:
import os
import h5py
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from ezc3d import c3d
from scipy.signal import resample
from scipy.io import loadmat, savemat
from joblib import load, dump
from tqdm import tqdm

In [2]:
PATH_DATASETS = '../../Dataset/'
PATH_DATASET_1 = 'gait-dbase-1/'

N_SAMPLES = 1024
FORCE_THRESHOLD = 20
TOE_HEIGTH_THRESHOLD = 40

In [3]:
path = os.path.join(PATH_DATASETS, PATH_DATASET_1)
subjects = os.listdir(path)

features = pd.DataFrame()
target = pd.DataFrame()

for subject in tqdm(subjects):
    files_path = os.path.join(path, subject)
    files = os.listdir(files_path)
    for filename in files:

        try:
            content = c3d(os.path.join(path, subject, filename), extract_forceplat_data=True)
            trajectory_x_l = content['data']['points'][1, 9:15, :]
            trajectory_y_l = content['data']['points'][1, 9:15, :]
            trajectory_z_l = content['data']['points'][2, 9:15, :]

            trajectory_x_r = content['data']['points'][1, 20:26, :]
            trajectory_y_r = content['data']['points'][1, 20:26, :]
            trajectory_z_r = content['data']['points'][2, 20:26, :]

            trajectory_x_l = resample(trajectory_x_l, N_SAMPLES, axis=1).T
            trajectory_y_l = resample(trajectory_y_l, N_SAMPLES, axis=1).T
            trajectory_z_l = resample(trajectory_z_l, N_SAMPLES, axis=1).T

            trajectory_x_r = resample(trajectory_x_r, N_SAMPLES, axis=1).T
            trajectory_y_r = resample(trajectory_y_r, N_SAMPLES, axis=1).T
            trajectory_z_r = resample(trajectory_z_r, N_SAMPLES, axis=1).T

            force_x_1 = content['data']['platform'][0]['force'][0, :]
            force_y_1 = content['data']['platform'][0]['force'][1, :]
            force_z_1 = content['data']['platform'][0]['force'][2, :]

            force_x_2 = content['data']['platform'][1]['force'][0, :]
            force_y_2 = content['data']['platform'][1]['force'][1, :]
            force_z_2 = content['data']['platform'][1]['force'][2, :]

            force_x_1 = resample(force_x_1, N_SAMPLES, axis=0).T
            force_y_1 = resample(force_y_1, N_SAMPLES, axis=0).T
            force_z_1 = resample(force_z_1, N_SAMPLES, axis=0).T

            force_x_2 = resample(force_x_2, N_SAMPLES, axis=0).T
            force_y_2 = resample(force_y_2, N_SAMPLES, axis=0).T
            force_z_2 = resample(force_z_2, N_SAMPLES, axis=0).T

            force_combined_z = force_z_1 + force_z_2
            force_mask = (force_combined_z >= FORCE_THRESHOLD)

            trajectory_x_l = trajectory_x_l[force_mask, :]
            trajectory_y_l = trajectory_y_l[force_mask, :]
            trajectory_z_l = trajectory_z_l[force_mask, :]

            trajectory_x_r = trajectory_x_r[force_mask, :]
            trajectory_y_r = trajectory_y_r[force_mask, :]
            trajectory_z_r = trajectory_z_r[force_mask, :]

            trajectory_x_l = resample(trajectory_x_l, N_SAMPLES, axis=0)
            trajectory_y_l = resample(trajectory_y_l, N_SAMPLES, axis=0)
            trajectory_z_l = resample(trajectory_z_l, N_SAMPLES, axis=0)

            trajectory_x_r = resample(trajectory_x_r, N_SAMPLES, axis=0)
            trajectory_y_r = resample(trajectory_y_r, N_SAMPLES, axis=0)
            trajectory_z_r = resample(trajectory_z_r, N_SAMPLES, axis=0)

            force_x_1 = force_x_1[force_mask]
            force_y_1 = force_y_1[force_mask]
            force_z_1 = force_z_1[force_mask]

            force_x_2 = force_x_2[force_mask]
            force_y_2 = force_y_2[force_mask]
            force_z_2 = force_z_2[force_mask]

            force_x_1 = resample(force_x_1, N_SAMPLES, axis=0)
            force_y_1 = resample(force_y_1, N_SAMPLES, axis=0)
            force_z_1 = resample(force_z_1, N_SAMPLES, axis=0)

            force_x_2 = resample(force_x_2, N_SAMPLES, axis=0)
            force_y_2 = resample(force_y_2, N_SAMPLES, axis=0)
            force_z_2 = resample(force_z_2, N_SAMPLES, axis=0)

            force_x_l = 0
            force_y_l = 0
            force_z_l = 0
            force_x_r = 0
            force_y_r = 0
            force_z_r = 0

            fm1_z_l = trajectory_z_l[:, 3]
            peak_height = np.max(fm1_z_l)
            force_mask_1 = (force_z_1 >= FORCE_THRESHOLD)
            area_of_interest = fm1_z_l[force_mask_1]

            if peak_height in area_of_interest:
                force_x_l = force_x_2
                force_x_r = force_x_1
                force_y_l = force_y_2
                force_y_r = force_y_1
                force_z_l = force_z_2
                force_z_r = force_z_1

            else:
                force_x_l = force_x_1
                force_x_r = force_x_2
                force_y_l = force_y_1
                force_y_r = force_y_2
                force_z_l = force_z_1
                force_z_r = force_z_2
            
            
            _features = pd.DataFrame(
                np.concatenate([trajectory_x_l, trajectory_y_l, trajectory_z_l], axis=1),
                columns=[
                    'lfal_x', 'ltam_x', 'lfcc_x', 'lfm1_x', 'lfm2_x', 'lfm5_x',
                    'lfal_y', 'ltam_y', 'lfcc_y', 'lfm1_y', 'lfm2_y', 'lfm5_y',
                    'lfal_z', 'ltam_z', 'lfcc_z', 'lfm1_z', 'lfm2_z', 'lfm5_z'
                ]
            )

            _target = pd.DataFrame(
                np.stack([force_x_r, force_y_r, force_z_r], axis=1),
                columns=['fx', 'fy', 'fz']
            )

            features = pd.concat([features, _features], axis=0)
            target = pd.concat([target, _target], axis=0)

        except Exception as e:
            # print('Error occured', e)
            pass



100%|██████████| 50/50 [03:31<00:00,  4.24s/it]


In [4]:
features

,lfal_x,ltam_x,lfcc_x,lfm1_x,lfm2_x,lfm5_x,lfal_y,ltam_y,lfcc_y,lfm1_y,lfm2_y,lfm5_y,lfal_z,ltam_z,lfcc_z,lfm1_z,lfm2_z,lfm5_z
0,315.805034,229.923447,268.002196,250.332418,307.221042,355.662087,315.805034,229.923447,268.002196,250.332418,307.221042,355.662087,97.109251,111.920361,70.391584,21.615834,35.859695,23.911881
1,315.558911,228.515445,269.036028,251.981771,308.870052,357.669964,315.558911,228.515445,269.036028,251.981771,308.870052,357.669964,88.033834,105.511873,57.503490,19.313654,34.930898,20.973876
2,316.752090,231.831312,267.820187,249.600376,306.444526,354.207775,316.752090,231.831312,267.820187,249.600376,306.444526,354.207775,105.485854,118.474152,82.123236,24.443153,37.467019,26.913194
3,316.535932,230.557797,268.766478,251.105227,307.949703,356.033867,316.535932,230.557797,268.766478,251.105227,307.949703,356.033867,97.279228,112.691002,70.463328,22.339615,36.618213,24.241545
4,316.642032,230.473286,269.069504,251.487472,308.325646,356.386207,316.642032,230.473286,269.069504,251.487472,308.325646,356.386207,96.156232,112.045142,68.827439,22.089813,36.593097,23.873021
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1019,337.375286,431.516871,389.610065,455.422109,399.987403,348.992187,337.375286,431.516871,389.610065,455.422109,399.987403,348.992187,171.593557,159.832156,192.172513,43.198414,39.212795,37.817205
1020,335.078082,429.143426,387.508124,456.526995,401.156275,349.827155,335.078082,429.143426,387.508124,456.526995,401.156275,349.827155,186.118487,169.248808,217.967915,38.437686,31.560761,34.156860
1021,334.221502,428.248941,386.791999,457.016610,401.621599,350.268135,334.221502,428.248941,386.791999,457.016610,401.621599,350.268135,192.058770,173.118944,228.407639,36.838866,28.691684,32.879637
1022,338.496536,432.628421,390.991688,455.285613,399.557418,349.284946,338.496536,432.628421,390.991688,455.285613,399.557418,349.284946,167.147134,157.028591,183.700972,46.499497,42.912168,40.069398


In [5]:
target

,fx,fy,fz
0,0.689221,2.345322,35.940089
1,1.152053,3.758829,60.123584
2,1.198161,3.739331,63.750031
3,1.212495,3.634767,66.205383
4,1.331818,3.936096,75.216480
...,...,...,...
1019,0.672235,0.479558,54.962165
1020,0.358541,0.284977,53.190843
1021,0.190486,0.161487,40.055678
1022,-0.010146,0.031341,22.115941


In [6]:
path = '../../Dataset/Processed/data.joblib'
data = {
    'X': features,
    'y': target
}
dump(data, path)

['../../Dataset/Processed/data.joblib']